# BiSRe-LLM: Perturbation Experiment with Mistral

In this notebook I set up the perturbation experiments for Mistral. First, the general infrastructure for interacting with Mistral is set up. Second, the prompt is designed as well as the iterative API requests to Huggingface to generate the answers to the according questions and answer options. In a third and last step, the answers of the model are stored in a dataframe for further analysis.

## Dataset

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

main_question = ["question"]
main_answer = ["list_answer_options"]

#bias_answer_columns = ["list_answer_no_refusal","list_answer_no_middle", "list_answer_middle", "list_answer_options_reversed", "list_answer_options", "original_personality_prime", "original_emotional_prime"]
#non_bias_question_columns = ["question_typo", "question_letter_swap","question_synonym","question_paraphrased", "question_keyboardtypo"]

# delayed experiments with middle category
bias_answer_columns = ["list_answer_middle"]
non_bias_question_columns = []

# read the preprocessed questionnaire
#questionnaire =  pd.read_excel('../assets/20241029_selected_questionnaire.xlsx', engine='openpyxl')
questionnaire =  pd.read_csv('../assets/20241029_selected_questionnaire.csv', sep=";")
# subset the questionnaire with the ten selected questions
questionnaire = questionnaire[questionnaire["selected"]==1]
print(questionnaire.shape)
questionnaire.head(2)

(62, 20)


,Unnamed: 0,question_ID,question,full_question_ID,list_answer_options,list_answer_no_refusal,len_answer_options,odd_answer_options,list_answer_no_middle,list_answer_middle,original_emotional_prime,original_personality_prime,list_answer_options_reversed,_merge,question_typo,question_letter_swap,question_synonym,question_paraphrased,question_keyboardtypo,selected
0,0,Q1,"Important in life: Family For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Family",Q1,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important ic life: Fimily nor eaca jf the following aspecto, indicete how important it ir wn your liae. Would bou say it is very imporcant, rathes importano, not very imporgant ob not impodtant at all? Famizy","Impartont in life: Flmiay For each of the following sapects, indicate how important it is in your life. uoWld you yas it is evry important, ratreh important, ton very important or not important ta all? Family","Crucial in life: Family For each of the following aspects, indicate how significant it is in your life. Would you say it is very important, rather important, not very important or not at all important? Family","How important is family to you? Please rate its significance in your life on a scale of ""very important"" to ""not important at all"".","Imporyant in kife: Family For esch of rhe following aspects, indicate how important ut is un your lide. Would you say it ia very imporrant, rather importamt, nit very important ir nor important ay all? Fanily",1
1,1,Q2,"Important in life: Friends For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Friends",Q2,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather Important ', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important ix life: Fpiends For xach of the following aspects, indicate how important iv iu in kour lifev Would dou say it os very importcnt, rather impoutant, nom vwry important hr not important xt bll? Friemds","Important ni :ifel Frisnde For each of the following aspects, indicaet 

## Experiments on Remote PC
The following section contains the experimental setup and scripts conducted on the remote PC.

In [3]:
%pip install mistral-inference
%pip install transformers
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install 'accelerate>=0.26.0'
%pip install bitsandbytes
%pip install sentencepiece

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
     ---------------------------------- ----- 8.9/10.2 MB 43.6 MB/s eta 0:00:01
     --------------------------------------- 10.2/10.2 MB 40.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      Traceback (most recent call last):
        File "c:\Users\JensMichaelRupprecht\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "c:\Users\JensMichaelRupprecht\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\JensMichaelRupprecht\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\JensMichaelRupprecht\AppD

  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   ----------------------------------- ---- 9.2/10.4 MB 45.6 MB/s eta 0:00:01
   ---------------------------------------- 10.4/10.4 MB 40.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 39.2 MB/s eta 0:00:00

   ---------------------------------------- 0/8 [tqdm]
   ---------------------------------------- 0/8 [tqdm]
   ---------------------------------------- 0/8 [tqdm]
   ---------------------------------------- 0/8 [tqdm]
   ---------------------------------------- 0/8 [tqdm]
   ---------------------------------------- 0/8 [tqdm]
   ----- ---------------------------------- 1/8 [safetensors]
   ---------- ----------------------------- 2/8 [regex]
   ---------- ----------------------------- 2/8 [regex]
   --------------- ------------------------ 3/8

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ------------------------------------- 536.2/536.2 kB 12.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 GB 55.0 MB/s eta 0:00:53
   ---------------------------------------- 0.0/2.9 GB 60.4 MB/s eta 0:00:48
    --------------------------------------- 0.0/2.9 GB 63.8 MB/s eta 0:00:45
    --------------------------------------- 0.1/2.9 GB 61.9 MB/s eta 0:00:47
    --------------------------------------- 0.1/2.9 GB 63.0 MB/s eta 0:00:46
   - -------------------------------------- 0.1/2.9 GB 65.6 MB/s eta 0:00:44
   - -------------------------------------- 0.1/2.9 GB 67.5 MB/s eta 0:00:42
   - -------------------------------------- 0.1/2.9 GB 68.1 MB/s eta 0:00:42
   - -------------------------------------- 0.1/2.9 GB 68.1 MB/s eta 0:00:41
   - ---------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'accelerate": Expected package name at the start of dependency specifier
    'accelerate
    ^


   ---------------------------------------- 0.0/75.4 MB ? eta -:--:--
   ---- ----------------------------------- 8.1/75.4 MB 42.3 MB/s eta 0:00:02
   --------- ------------------------------ 17.6/75.4 MB 44.8 MB/s eta 0:00:02
   -------------- ------------------------- 27.0/75.4 MB 44.6 MB/s eta 0:00:02
   ------------------ --------------------- 35.4/75.4 MB 43.7 MB/s eta 0:00:01
   ----------------------- ---------------- 44.0/75.4 MB 43.6 MB/s eta 0:00:01
   ---------------------------- ----------- 53.0/75.4 MB 43.5 MB/s eta 0:00:01
   --------------------------------- ------ 62.4/75.4 MB 43.8 MB/s eta 0:00:01
   -------------------------------------- - 72.1/75.4 MB 44.1 MB/s eta 0:00:01
   ---------------------------------------  75.2/75.4 MB 43.6 MB/s eta 0:00:01
   ---------------------------------------- 75.4/75.4 MB 39.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
    

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      Traceback (most recent call last):
        File "c:\Users\JensMichaelRupprecht\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "c:\Users\JensMichaelRupprecht\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\JensMichaelRupprecht\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\JensMichaelRupprecht\AppD

In [1]:
# Just load once
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
# llm_int8_enable_fp32_cpu_offload=True,
# bnb_8bit_quant_type='nf4',
# bnb_8bit_use_double_quant=True,
# bnb_8bit_compute_dtype=torch.bfloat16,
load_in_8bit=True
)

model_8bit = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    quantization_config=quantization_config
)

c:\Users\JensMichaelRupprecht\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\JensMichaelRupprecht\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\JensMichaelRupprecht\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2", device=0)
pipe(messages, max_new_tokens=10)

Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.32s/it]
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': ' I am a large language model trained by Mistral'}]}]

In [4]:
messages = [
    {"role": "user", "content": "How is the wheather today?"},
]
pipe(messages, max_new_tokens=1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'How is the wheather today?'},
   {'role': 'assistant', 'content': ' I'}]}]

In [ ]:
# create empty response list which is transformed to a dataframe

response_list = []

# define rounds to be tested
rounds = 25

for round in range(0,rounds):
  print("Current round: ", round)
  torch.cuda.empty_cache()
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      for i,k in enumerate(bias_answer_columns):
          print(bias_answer_columns)
          # first retrieve original question
          question = questionnaire.iloc[row]["question"]
          # second retrieve bias perturbed answer options from the questionnaire
          answer_options = questionnaire.iloc[row][k]
          messages = [{"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]
          answer = pipe(messages, max_length=1000, max_new_tokens=1)
          response = answer[0]["generated_text"][1]["content"]
          print(response)
          
          df_response_input = [round,question_ID, question, answer_options, k,  response]
          response_list.append(df_response_input)
          torch.cuda.empty_cache()
      for p,n in enumerate(non_bias_question_columns):
              # first retrieve non-bias perturbed question formats
              question = questionnaire.iloc[row][n]
              # second retrieve original answer option scale
              answer_options = questionnaire.iloc[row]["list_answer_options"]
              messages = [{"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]
              answer = pipe(messages, max_length=1000, max_new_tokens=1)
              response = answer[0]["generated_text"][1]["content"]
              print(response)
              df_response_input = [round,question_ID, question, answer_options, n, response]
              response_list.append(df_response_input)
              torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=1) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Current round:  0
['list_answer_middle']


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=1) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 I
['list_answer_middle']


In [13]:
print("Length of responses: ", len(response_list))

Length of responses:  1550


In [14]:
# create a bias interaction experiment where there is a question-side and answer option-side bias
# here we use
perturbed_question = "list_answer_options_reversed"
perturbed_question = "list_answer_options_reversed"
perturbed_answer = "question_paraphrased"

for round in range(0,rounds):
  print("Current round: ", round)
  torch.cuda.empty_cache()
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      # first retrieve original question
      question = questionnaire.iloc[row]["question_paraphrased"]
      # second retrieve bias perturbed answer options from the questionnaire  
      answer_options = questionnaire.iloc[row]["list_answer_options_reversed"]      
      messages = [{"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]    
      answer = pipe(messages, max_length=1000, max_new_tokens=10)       
      response = answer[0]["generated_text"][1]["content"]         
      print(response)  
      df_response_input = [round,question_ID, question, answer_options, "interaction_paraphrase_reversed",  response]
      response_list.append(df_response_input)
      torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Current round:  0


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 1=Very important


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 '10=Essential feature'


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 5=Definitely essential


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 10 (absolutely essential) for the
 '1=Not an essential feature'


In [15]:
print("Length of responses after interaction experiments: ", len(response_list))

Length of responses after interaction experiments:  3100


In [16]:
# create an empty dataframe which is filled with the model responses to the q&a combinations
response_columns = ["round","question_ID", "question", "answer_options", "type", "response"]
model_response = pd.DataFrame(response_list, columns=response_columns)
model_response["model_name"] = "Mistral-7B-Instruct-v0.3"

In [17]:
from datetime import datetime

def get_date_string():
    # Get the current date
    now = datetime.now()
    
    # Format the date as YYYYMMDD
    date_string = now.strftime('%Y%m%d')
    
    return date_string

current_date = get_date_string()
model_response.to_csv(f"../assets/final_interviews/{current_date}_mistral_7B_selected_interviews.csv", encoding="utf-8", sep=";")